In [ ]:
from evotorch import Problem
from evotorch.algorithms import GeneticAlgorithm
from evotorch.decorators import vectorized
from evotorch.operators import SimulatedBinaryCrossOver

import torch

In [ ]:
@vectorized
def fitness(x: torch.Tensor) -> torch.Tensor:
    return torch.linalg.norm(x, dim=-1)

In [ ]:
problem = Problem(
    "min",
    fitness,
    solution_length=5,
    bounds=(
        [-1, -2, -100, -1000, 10.0],
        [1, 2, 100, 1000, 10.5],
    ),
    dtype=torch.float32,
)

problem

In [ ]:
def my_manual_mutation(x: torch.Tensor) -> torch.Tensor:
    # The default GaussianMutation of EvoTorch does not (yet) support different standard deviation values
    # per variable. However, we can define our own mutation operator which adds noise of different magnitudes
    # to different variables, like shown here:
    [_, solution_length] = x.shape
    dtype = x.dtype
    device = x.device

    # Generate Gaussian noise where each column has its own magnitude
    noise = (
        torch.randn(solution_length, dtype=dtype, device=device)
        * torch.tensor([1, 2, 100, 1000, 0.1], dtype=dtype, device=device)
    )

    result = x + noise

    # Because this is a manual operator, we need to specify how it should respect the strict boundaries.
    # In the case of this example, we just clip the values.
    result = torch.minimum(result, problem.upper_bounds)
    result = torch.maximum(result, problem.lower_bounds)

    return result

In [ ]:
searcher = GeneticAlgorithm(
    problem,
    popsize=100,
    operators=[
        SimulatedBinaryCrossOver(problem, tournament_size=4, eta=8),
        my_manual_mutation,
    ],
)
searcher

In [ ]:
searcher.step()  # Take just one step. Just to see how the population looks like after one generation.

In [ ]:
list(searcher.population[:10])